In [ ]:
import xarray as xr

file_path = "/data3/interns/NRT_CO2_Emission_Map_Project/HaoHu_work/ERA5_DT2/data.grib"

ds = xr.open_dataset(file_path, engine="cfgrib")

print(ds)

print("Variables:", list(ds.data_vars))

print("Dimensions:", ds.dims)
print("Coordinates:", ds.coords)

# Resample

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

template_path = "/data3/interns/NRT_CO2_Emission_Map_Project/HaoHu_work/XCO2_resample/global_grid_0.1_2019_2025.nc"
ds_template = xr.open_dataset(template_path)
target_lat = ds_template["lat"]
target_lon = ds_template["lon"]
target_time = ds_template["time"]

output_folder = "/data3/interns/NRT_CO2_Emission_Map_Project/HaoHu_work/ERA5_resample"
os.makedirs(output_folder, exist_ok=True)

variables = {
   "T2": True,
   "TSK": True,
   "U10": False,
   "V10": False,
   "TotalColumnWater": False,
   "SurfacePressure": False,
   "MeanSeaLevelPressure": False,
   "PBLH": False,
   "DT2": False
}

var_name_map = {
    "T2": "t2m",
    "U10": "u10",
    "V10": "v10",
    "TSK": "skt",
    "Pr": "tp", 
    "SurfacePressure": "sp",
    "MeanSeaLevelPressure": "msl",
    "PBLH": "blh",
    "SurfaceNetSolarRadiation": "ssr",
    "SurfaceNetThermalRadiation": "str",
    "TotalColumnWater": "tcw",
    "Evaporation": "e",
    "DT2": "d2m" 
}

def extend_data_for_spherical_interp(data):
    data = data.assign_coords(
        longitude=((data.longitude + 180) % 360) - 180
    )
    
    data_left = data.assign_coords(longitude=(data.longitude - 360))
    data_right = data.assign_coords(longitude=(data.longitude + 360))
    data_lon_extended = xr.concat([data_left, data, data_right], dim="longitude")

    data_north = data_lon_extended[::-1].assign_coords(
        latitude=(180 - data_lon_extended.latitude)
    )
    data_south = data_lon_extended[::-1].assign_coords(
        latitude=(-180 - data_lon_extended.latitude)
    )

    data_extended = xr.concat([data_south, data_lon_extended, data_north], dim="latitude")

    data_extended = data_extended.sortby(["latitude", "longitude"])
    data_extended = data_extended.sel(
        latitude=~data_extended.latitude.to_index().duplicated(),
        longitude=~data_extended.longitude.to_index().duplicated()
    )
    
    return data_extended


for var_name, convert_K_to_C in variables.items():
    print(f"\n🌡️ Process：{var_name}")

    folder_path = f"/data3/interns/NRT_CO2_Emission_Map_Project/HaoHu_work/ERA5_{var_name}"
    data_path = os.path.join(folder_path, "data.grib")
    
    ds_era5 = xr.open_dataset(data_path, engine="cfgrib", chunks={"time": 24})
    
    grib_var = var_name_map.get(var_name, None)

    data = ds_era5[grib_var]

    if convert_K_to_C:
        data = data - 273.15

    if var_name == "Pr":
        data_daily = data.resample(time="1D").sum()
    else:
        data_daily = data.resample(time="1D").mean()

    results = []

    for day in tqdm(target_time.values, desc=f"📆 Interplote {var_name}", unit="day"):
        date_str = str(pd.to_datetime(day).date())
        day_data = data_daily.sel(time=day)

        extended_data = extend_data_for_spherical_interp(day_data)
        interp = extended_data.interp(
            latitude=target_lat,
            longitude=target_lon,
            method="linear"
        )

        interp = interp.assign_coords(
            lat=interp.latitude,
            lon=interp.longitude
        ).drop_vars(["latitude", "longitude"])

        interp = interp.expand_dims(dim={"time": [day]})
        results.append(interp.astype(np.float32))

    data_all = xr.concat(results, dim="time")
    drop_coords = [c for c in ['number', 'step', 'surface'] if c in data_all.coords]
    data_all = data_all.drop_vars(drop_coords)
    
    grib_var = var_name_map.get(var_name, None)
    out_file = os.path.join(output_folder, f"{grib_var}_daily_0p1deg.nc")
    data_all.to_netcdf(out_file)

